In [ ]:
import cv2
import numpy as np
import time
from IPython.display import clear_output
import math

WIND_X = 800
WIND_Y = 600

GOAL_POS = 100
GOAL_LEN = 300

FIELD_COLOR = (10,  240, 10)
GOAL_COLOR  = (30,   40, 70)
BALL_COLOR  = (120, 130, 30)
ROBOT_COLOR = (160,  40, 10)
TRAJ_COLOR  = (60,   40, 180)

BALL_RADIUS   = 20
CIRCLE_RADIUS = 50
ROBOT_SIZE    = 50

def draw_scenario (xr, yr, xb, yb, trajectory, yaw, command):
    canvas = np.zeros ((WIND_Y, WIND_X, 3), np.uint8)
    
    cv2.rectangle (canvas, (0, 0), (WIND_X, WIND_Y), FIELD_COLOR, thickness = -1)
    
    cv2.circle    (canvas, (xb, yb), CIRCLE_RADIUS, GOAL_COLOR)
    cv2.circle    (canvas, (xb, yb), BALL_RADIUS, BALL_COLOR, -1)
    
    robot_tl = (xr - int (ROBOT_SIZE / 2), yr - int (ROBOT_SIZE / 2))
    robot_br = (xr + int (ROBOT_SIZE / 2), yr + int (ROBOT_SIZE / 2))
    cv2.rectangle (canvas, robot_tl, robot_br, GOAL_COLOR, -1)
    
    cv2.line (canvas, (WIND_X, GOAL_POS), (WIND_X, GOAL_POS + GOAL_LEN), GOAL_COLOR, 5)
    
    cv2.line (canvas, (xr, yr), (xr + int(50 * math.cos(yaw)), yr + int(50 * math.sin(yaw))), ROBOT_COLOR, 3)
    
    for i in range (len (trajectory) - 1):
        cv2.line (canvas, trajectory [i], trajectory [i + 1], TRAJ_COLOR, 3)
        
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10, WIND_Y - 20)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2

    cv2.putText(canvas, command, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    
    return canvas


def find_trajectory_test (xr, yr, xb, yb):
    traj = []
    
    traj.append ((xr, yr))
    traj.append ((xb, yb))
    
    return traj

def find_trajectory_basic (xr, yr, xb, yb, max_step):
    traj = []
    
    traj.append ((xr, yr))
    
    #find starting point on the circle
    rbx = xb - xr #robot-ball x
    rby = yb - yr #robot-ball y
    
    length_rb = int (math.sqrt (rbx**2 + rby**2))
    
    spx = xb - int (CIRCLE_RADIUS * rbx / length_rb) #start point x
    spy = yb - int (CIRCLE_RADIUS * rby / length_rb) #start point y
    
    if (abs (xb - xr) < abs (yb - yr)):
        traj.append ((spx, spy))
    
    #find kick point on the circle
    gbx = xb - WIND_X                        #goal-ball x
    gby = yb - GOAL_POS - int (GOAL_LEN / 2) #goal-ball y
    
    length_gb = int (math.sqrt (gbx**2 + gby**2))
    
    kpx = xb + int (CIRCLE_RADIUS * gbx / length_gb) #kick point x
    kpy = yb + int (CIRCLE_RADIUS * gby / length_gb) #kick point y
    
    traj.append ((kpx, kpy))
    traj.append ((WIND_X, GOAL_POS + int (GOAL_LEN / 2)))
        
    return traj

def find_trajectory (xr, yr, xb, yb, max_step):
    traj = []
    
    traj.append ((xr, yr))
    
    #-----------------------------------------------------------
    #find starting point on the circle
    
    xbr = xb - xr #x ball relative
    ybr = yb - yr #y ball relative
    
    r    = CIRCLE_RADIUS
    leng = math.sqrt (xbr**2 + ybr**2)
    
    beta  = np.arcsin (float (ybr) / leng)
    alpha = np.arcsin (float (r) / leng)
    
    if (xb < xr):# + CIRCLE_RADIUS):
        sx = 0
        sy = 0
        
        if (yr + (yr - GOAL_POS - int (GOAL_LEN / 2)) * (xr - xb) / (WIND_X - xb) > yb):
            sx = - leng * math.cos (alpha + beta) * math.cos (alpha) + xr
            sy = leng * math.sin (alpha + beta) * math.cos (- alpha) + yr
        
        else:
            alpha = - alpha
            
            sx = - leng * math.cos (alpha + beta) * math.cos (alpha) + xr
            sy = leng * math.sin (alpha + beta) * math.cos (- alpha) + yr
        
        traj.append ((int (sx), int (sy)))
    
    #-----------------------------------------------------------
    #find kick point on the circle
    gbx = xb - WIND_X                        #goal-ball x
    gby = yb - GOAL_POS - int (GOAL_LEN / 2) #goal-ball y
    
    length_gb = int (math.sqrt (gbx**2 + gby**2))
    
    kpx = xb + int (CIRCLE_RADIUS * gbx / length_gb) #kick point x
    kpy = yb + int (CIRCLE_RADIUS * gby / length_gb) #kick point y
    
    traj.append ((kpx, kpy))
    
    #move with proper steps on the circle
    
    #if (xr < xb):
    #    traj.append ((xr, yr))
    #    traj.append ((xb - CIRCLE_RADIUS, yb))
    
    #else:
    #    if (yr < yb):
    #    else:
    
    traj.append ((WIND_X, GOAL_POS + int (GOAL_LEN / 2)))
    
    return traj

def lin_trans(W, shift, vec):
    res = [vec[i] - shift[i] for i in range(2)]
    res = [int(W[i][0] * res[0] + W[i][1] * res[1]) for i in range(2)]
    return res

def convert_trajectory(traj, yaw):   
    shift = traj[0]
    rotmat = [[math.cos(yaw), math.sin(yaw)], [-1 * math.sin(yaw), math.cos(yaw)]]
    
    rtraj = [lin_trans(rotmat, shift, point) for point in traj]
    
    return rtraj

def cross_dot(v1, v2):
    return -int(v1[0] * v2[1] - v1[1] * v2[0]), int(v1[0] * v2[0] + v1[1] * v2[1])

def make_decision(rtraj, min_dist, ang_thres1, ang_thres2):
    
    targvec = (rtraj[1][0] - rtraj[2][0], rtraj[1][1] - rtraj[2][1])
    tv_ln = math.sqrt(targvec[0] ** 2 + targvec[1] ** 2)
    norm = (int(20 * targvec[1] / tv_ln), int(-20 * targvec[0] / tv_ln))
    rtraj[1][0] -= norm[0]
    rtraj[1][1] -= norm[1]
    
    path = rtraj[1]
    
    vec1 = path
    vec1_ln = math.sqrt(vec1[0] ** 2 + vec1[1] ** 2)
    vec2 = (rtraj[2][0] - vec1[0], rtraj[2][1] - vec1[1])
    vec2_ln = math.sqrt(vec2[0] ** 2 + vec2[1] ** 2)
    
    prod = cross_dot(vec1, vec2)
    ang2 = prod[0] / vec1_ln / vec2_ln
    
    if prod[1] > 0:
        ang2 = (math.pi - abs(ang2)) * prod[0] / abs(prod[0])
        
    
    if vec1_ln < min_dist:
        
        if path[0] <= 0:
            return "problem"
        
        if ang2 > 0 and (math.pi - abs(ang2)) > ang_thres2:
            return "step right"
        
        elif ang2 < 0 and (math.pi - abs(ang2)) > ang_thres2:
            return "step left"
        
        if path[0] > 0:
            if abs(math.atan(path[1] / path[0])) < ang_thres1:
                return "strike"
            elif path[1] < 0:
                return "turn left"
            else:
                return "turn right"
        else:
            if path[1] < 0:
                return "turn left"
            else:
                return "turn right"

    elif path[0] > 0:
        if abs(math.atan(path[1] / path[0])) < ang_thres1:
            return "step forward"
        elif path[1] < 0:
            return "turn left"
        else:
            return "turn right"
    else:
        if path[1] < 0:
            return "turn left"
        else:
            return "turn right"
    

In [ ]:
cv2.namedWindow  ("movement", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("movement", WIND_Y, WIND_X)

xr = int (1 * WIND_X / 5)
yr = int (WIND_Y / 3)

xb = int (WIND_X / 2)
yb = int (WIND_Y / 2)

yaw = 0

upd = True

trajectory = []

while (True):    
    cv2.waitKey (1)
    
    if (upd == True):
        trajectory = find_trajectory (xr, yr, xb, yb, 20)
        rtraj = convert_trajectory(trajectory, yaw)
        command = make_decision(rtraj, 20, math.pi / 20, math.pi / 20)
        upd = False
    
    canvas = draw_scenario (xr, yr, xb, yb, trajectory, yaw, command)
            
    cv2.imshow ("movement", canvas)
    
    time.sleep  (0.01)
    
    #handle keyboard events
    keyb = cv2.waitKey (1)
    
    if (keyb != -1):
        upd = True
    
    if (keyb & 0xFF == ord ('q')):
        break

    elif (keyb & 0xFF == ord ('t')):
        yr -= 5
    
    elif (keyb & 0xFF == ord ('f')):
        xr -= 5
    
    elif (keyb & 0xFF == ord ('g')):
        yr += 5
    
    elif (keyb & 0xFF == ord ('h')):
        xr += 5

    elif (keyb & 0xFF == ord ('i')):
        yb -= 5
    
    elif (keyb & 0xFF == ord ('j')):
        xb -= 5
    
    elif (keyb & 0xFF == ord ('k')):
        yb += 5
    
    elif (keyb & 0xFF == ord ('l')):
        xb += 5
        
    elif (keyb & 0xFF == ord ('z')):
        yaw -= 0.1
        
    elif (keyb & 0xFF == ord ('c')):
        yaw += 0.1
        
    elif (keyb & 0xFF == ord ('w')):
        xr += int(5 * math.cos(yaw))
        yr += int(5 * math.sin(yaw))
        
    elif (keyb & 0xFF == ord ('a')):
        xr += int(5 * math.sin(yaw))
        yr += int(-5 * math.cos(yaw))
    
    elif (keyb & 0xFF == ord ('d')):
        xr += int(-5 * math.sin(yaw))
        yr += int(5 * math.cos(yaw))
    

cv2.waitKey           (1)
cv2.destroyAllWindows ()

In [87]:
import cv2
import numpy as np
import time
from IPython.display import clear_output
import math
import requests
import time

#TODO: свитч на кнопки, словарь с активностями
#TODO: отрефакторить это произведение искусства
#TODO: поменять клавиатурный интерфейс хождения
#TODO: словарь на действия и кнопки, а то совсем плохо
#TODO: добавить хэндлинг русских букв
#TODO: добавить флаг на работу в автономном режиме, без робота
#TODO: добавить кнопки на: руку вперед, "привет", разжать/сжать пальцы, танец

WIND_X = 200
WIND_Y = 200




cv2.namedWindow  ("remote_controller", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("remote_controller", (WIND_Y, WIND_X))

ip_prefix  = "http://"

#ip_num = "10.197.241.216"
# ip_num = "10.0.0.102"
ip_num = "192.168.1.249"
# ip_num = "192.168.43.65"
# ip_num = "192.168.1.29"

#ip_num = "192.168.43.42"
ip_postfix = ":"

ip = ip_prefix + ip_num + ip_postfix

port = "9191"

canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 100

hello = [        #очень приятно
         "/?action=/stand&text=qwer",
        "/?action=/rest&text=qwer",
        "/?action=/stand&text=qwer"]

activity1 = ["/?action=/play_water&text=qwer",
             "/?action=/hands_sides&text=open_right",
             "/?action=/stand&text=qwer",
             "/?action=/play_dog_breathe&text=qwer",
             "/?action=/hands_sides&text=open_right",
             "/?action=/stand&text=qwer",
             "/?action=/play_dog_cry&text=qwer",
             "/?action=/hands_sides&text=open_right",
             "/?action=/stand&text=qwer",
             "/?action=/play_parrot_2&text=qwer"
            ] # повторяй за мной руки в бок

activity2 = ["/?action=/play_car&text=qwer",
             "/?action=/play_airplane_2&text=qwer",            
             "/?action=/hands_front&text=qwer",
             "/?action=/stand&text=qwer"
            ] # меня создали люди 

activity3 = ["/?action=/play_cat_eating&text=qwer",
            "/?action=/play_cat_meow&text=qwer",
             "/?action=/play_dog_bark&text=qwer",
             ] # как твои дела мои хорошо

activity4 = ["/?action=/play_dog_breathe&text=qwer",
             "/?action=/hands_front&text=qwer",
             "/?action=/stand&text=qwer",
             "/?action=/play_dog_cry&text=qwer",
             "/?action=/hands_front&text=qwer",
             "/?action=/stand&text=qwer",
             "/?action=/play_parrot_2&text=qwer",
            ] # руки вперед




activity5 = ["/?action=/play_dog_bark&text=qwer",
             "/?action=/hands_front&text=qwer"]




activity6 = ["/?action=/play_zhel_tuk_tok&text=qwer",
             "/?action=/play_zhel_zayka&text=qwer",
             "/?action=/play_zhel_logo_zha_zhu&text=qwer",
             "/?action=/play_zhel_logo_ku_gi&text=qwer",
             "/?action=/dance&text=qwer"]

ip1 = "http://192.168.1.29:9562"
ip2 = "http://192.168.1.249:9191"

activity7 = [ip1 + "/?action=/stand&text=qwer",
             ip2 + "/?action=/stand&text=qwer",
             ip1 + "/?action=/right_hand_front&text=qwer",
             ip2 + "/?action=/right_hand_front&text=qwer",
             ip1 + "/?action=/rest&text=qwer",
             ip2 + "/?action=/rest&text=qwer"]
#http://192.168.1.249:9191/?action=/stand&text=r

queue  = []
queue_ = []

to_next_operation = True

time_ = 0

free_counter     = 0
max_free_counter = 10

curr_time = time.time ()
time_of_prev_press = 0.0

r = "66666666666666666666666"

dict_of_commands = {'z': '/?action=/stand&text=qwer', # встать
                   'w': '/?action=/dance&text=qwer', # танцевать
                   'e': '/?action=/right_hand_front&text=qwer', # руки вперед
                   'r': '/?action=/hands&text=open_right', # руки вбок
                   't': '/?action=/hands&text=close_right', # закрыть правую руку
                   'f': '/?action=/play_water&text=qwer', # смотри как я умею
                   'x': '/?action=/sit&text=qwer', # сесть
                   'c': '/?action=/rest&text=qwer', # корточки
                   'v': '/?action=/play_dog_breathe&text=qwer', # повторяй за мной
                   'b': '/?action=/rot_50&text=qwer', # повернуться на 50 влево
#                     '/?action=/play_airplane_1&text=qwer', # привет я нао
                   'y': '/?action=/play_dog_cry&text=qwer', # попробуй еще
                   'g': '/?action=/play_parrot_2&text=qwer', # молодец
                   'h': '/?action=/hello', # привет
                   'm': '/?action=/walk_m30&text=qwer', # назад 30
                   'j': '/?action=/rot_20&text=qwer', # повернуться на 20 влево
                   'u': '/?action=/walk_50&text=qwer', # вперед 50
                   'i': '/?action=/walk_20&text=qwer', # вперед 20
                   'k': '/?action=/rot_m20&text=qwer', # 
                   'l': '/?action=/rot_m50&text=qwer', # 
                   'p': '/?action=/hands&text=open_right', # открыть правую руку
                    '1': '/?action=/M1',
                    '2': '/?action=/M2',
                    '3': '/?action=/M3',
                    '4': '/?action=/M4',
                    '5': '/?action=/M5',
                    '6': '/?action=/M6',
                   }




def list_and_dict_with_ord (dict_of_commands):
    list_of_keys = []
    dict_with_ord = {}
    for i in list(dict_of_commands.keys()):
        list_of_keys.append(ord(i))
        dict_with_ord.update({ord(i): dict_of_commands[i]})
    return list_of_keys, dict_with_ord

list_of_keys, dict_with_ord = list_and_dict_with_ord(dict_of_commands)
    
mode_without_queue = True

while (True):    
    cv2.waitKey (1)
    
    curr_time = time.time ()
    
    cv2.imshow ("remote_controller", canvas)
    
    time.sleep  (0.03)
    
    #handle keyboard events
    keyb = cv2.waitKey (1)
    
#     #action = 'free'
#     text = 'qwer'
#     #r = requests.get (ip + port + "/" + "?" + "action=/" + action + "&" + "text=" + text)
#     #print ("status ", int (str (r) [13:14]))
    
#     free = 0#int (str (r) [13:14]) #6 free, 7 not free; don't ask, don't tell
     
#     if (free == 6 and len (queue) != 0):
#         #clear screen
#         clear_output (wait=True)
        
#         print ("next: ", queue [0], len (queue_) - len (queue) + 1, "/", len (queue_))
        
#         if (to_next_operation == True):        
#             requests.get (ip + port + queue [0])
#             queue.remove (queue [0])
#             to_next_operation = False
    
#     action = 'free'
#     text = 'qwer'
    
    free_counter += 1
    
#     if free == 7 or len(queue) > 0:
#         if (free_counter == max_free_counter):
#             free_counter = 0
#             r = requests.get (ip + port + "/" + "?" + "action=/" + action + "&" + "text=" + text)
    
#     #print ("status ", int (str (r) [13:14]))
    
#     free = int (str (r) [13:14]) #6 free, 7 not free; don't ask, don't tell
    
#     free = 6
    
    if (len (queue) != 0 and to_next_operation == True):
        #action = 'free'
        #text = 'qwer'
        #r = requests.get (ip + port + "/" + "?" + "action=/" + action + "&" + "text=" + text)
        free = 6#int (str (r) [13:14]) #6 free, 7 not free; don't ask, don't tell
        
        print (r)
        
        if free == 6:
            time.sleep(0.1)
            
            requests.get (ip + port + queue [0])
            print (queue [0])
            #requests.get (queue [0])
            
            queue.remove (queue [0])
            if not mode_without_queue:
                to_next_operation = False
    
    
#     if (free == 6 and len (queue) != 0 and to_next_operation == True):
#         time.sleep(0.5)
#         requests.get (ip + port + queue [0])
#         queue.remove (queue [0])
#         if not mode_without_queue:
#             to_next_operation = False
    
    
    if (keyb != -1):
        upd = True
    
    if (keyb & 0xFF == ord ('q')):
        break
    
    elif (keyb & 0xFF == ord(' ')):
        mode_without_queue = not mode_without_queue
    
    elif (keyb & 0xFF == ord ('s')):
        queue [:] = []
        r = requests.get (ip + port + "/?action=/stop&text=qwer") # остановить действие

    elif (keyb & 0xFF == ord ('a')):
        queue  [:] = []
        queue_ [:] = []
    
    #elif (keyb & 0xFF == ord ('b')):
    #    queue [:] = []
    #    queue [:] = activity7.copy ()
    #    print ("mde")
    
    elif (keyb in list_of_keys):
#         print(keyb)
        queue.append(dict_with_ord[keyb])
    
#     elif (keyb & 0xFF == ord ('d')):
#         if (keyb & 0xFF == ord('z')):
#             print('123')
    
    elif (keyb & 0xFF == ord ('0')):
        #if (curr_time - time_of_prev_press > 0.4):
        #    time_of_prev_press = curr_time
        to_next_operation = True
        
        
    
    
#     elif (keyb & 0xFF == ord ('z')):
#         r = requests.get (ip + port + "/?action=/stand&text=qwer") # встать
    
#     elif (keyb & 0xFF == ord ('x')):
#         r = requests.get (ip + port + "/?action=/sit&text=qwer") # сесть 
    
#     elif (keyb & 0xFF == ord ('c')):
#         r = requests.get (ip + port + "/?action=/rest&text=qwer") # корточки

    

#     elif (keyb & 0xFF == ord ('w')):
#         r = requests.get (ip + port + "/?action=/dance&text=qwer") # танцевать

#     elif (keyb & 0xFF == ord ('e')):
#         r = requests.get (ip + port + "/?action=/hands_front&text=qwer") # руки вперед

#     elif (keyb & 0xFF == ord ('r')):
#         r = requests.get (ip + port + "/?action=/hands_sides&text=open_right") # руки вбок

    #elif (keyb & 0xFF == ord ('e')):
    #    r = requests.get (ip + port + "/?action=/right_hand_front&text=qwer")

    #elif (keyb & 0xFF == ord ('r')):
    #    r = requests.get (ip + port + "/?action=/hands&text=open_right")


#     elif (keyb & 0xFF == ord ('t')):
#         r = requests.get (ip + port + "/?action=/hands&text=close_right") # закрыть правую руку
    
#     elif (keyb & 0xFF == ord ('p')):
#         r = requests.get (ip + port + "/?action=/hands&text=open_right") # открыть правую руку
    
#     elif (keyb & 0xFF == ord ('i')):
#         r = requests.get (ip + port + "/?action=/walk_20&text=qwer") # вперед два-три шага
            
#     elif (keyb & 0xFF == ord ('u')):
#         r = requests.get (ip + port + "/?action=/walk_50&text=qwer") # вперед 5 шагов

#     elif (keyb & 0xFF == ord ('m')):
#         r = requests.get (ip + port + "/?action=/walk_m30&text=qwer") # назад 4 шага
        
#     elif (keyb & 0xFF == ord ('h')):
#         r = requests.get (ip + port + "/?action=/rot_50&text=qwer") # повернуться на 50 влево

#     elif (keyb & 0xFF == ord ('j')):
#         r = requests.get (ip + port + "/?action=/rot_20&text=qwer") # повернуться на 20 влево

#     elif (keyb & 0xFF == ord ('k')):
#         r = requests.get (ip + port + "/?action=/rot_m20&text=qwer") # 

#     elif (keyb & 0xFF == ord ('l')):
#         r = requests.get (ip + port + "/?action=/rot_m50&text=qwer") #
        
        
#     elif (keyb & 0xFF == ord ('b')):
#         r = requests.get (ip + port + "/?action=/play_airplane_1&text=qwer") # привет я нао
        
    
#     elif (keyb & 0xFF == ord ('v')):
#         r = requests.get (ip + port + "/?action=/play_dog_breathe&text=qwer") # повторяй за мной
#     elif (keyb & 0xFF == ord ('y')):
#         r = requests.get (ip + port + "/?action=/play_dog_cry&text=qwer") #попробуй еще
#     elif (keyb & 0xFF == ord ('g')):
#         r = requests.get (ip + port + "/?action=/play_parrot_2&text=qwer") # молодец
#     elif (keyb & 0xFF == ord ('f')):
#         r = requests.get (ip + port + "/?action=/play_water&text=qwer") # смотри как я умею
    
#  #play_train
    
#     elif (keyb & 0xFF == ord ('0')):
#         #if (curr_time - time_of_prev_press > 0.4):
#         #    time_of_prev_press = curr_time
#         to_next_operation = True
    
#     if (len (queue) == 0):
#         if (keyb & 0xFF == ord ('1')):
#             queue [:] = []
#             print (queue)
#             queue = activity1.copy ()

#         elif (keyb & 0xFF == ord ('2')):
#             queue  [:] = []
            
#             queue  = activity2.copy ()

#         elif (keyb & 0xFF == ord ('3')):
#             queue  [:] = []
            
#             queue  = activity3.copy ()

#         elif (keyb & 0xFF == ord ('4')):
#             queue  [:] = []
            
#             queue  = activity4.copy ()

#         elif (keyb & 0xFF == ord ('5')):
#             queue  [:] = []
#             queue_ [:] = []
            
#             queue  = activity5.copy ()

#         elif (keyb & 0xFF == ord ('6')):
#             queue  [:] = []
            
#             queue = activity6.copy ()

    curr_time = time.time ()

#     print(curr_time - time_)

    time_ = curr_time

cv2.waitKey           (1)
cv2.destroyAllWindows ()

In [15]:
cv2.namedWindow  ("movement", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("movement", WIND_Y, WIND_X)

xr = int (1 * WIND_X / 5 + 50)
yr = int (WIND_Y / 3 + 150)

xb = int (WIND_X / 2 + 50)
yb = int (WIND_Y / 2 - 100)

yaw = 0

upd = True

trajectory = []

while (True):    
    cv2.waitKey (1)
    
    if (upd == True):
        trajectory = find_trajectory (xr, yr, xb, yb, 20)
        rtraj = convert_trajectory(trajectory, yaw)
        command = make_decision(rtraj, 20, math.pi / 20, math.pi / 20)
        upd = False
    
    canvas = draw_scenario (xr, yr, xb, yb, trajectory, yaw, command)
            
    cv2.imshow ("movement", canvas)
    
    time.sleep  (0.1)
    
    if command == "step forward":
        upd = True
        xr += int(5 * math.cos(yaw))
        yr += int(5 * math.sin(yaw))
    elif command == "turn left":
        upd = True
        yaw -= 0.1
    elif command == "turn right":
        upd = True
        yaw += 0.1
    elif command == "step left":
        upd = True
        xr += int(5 * math.sin(yaw))
        yr += int(-5 * math.cos(yaw))
    elif command == "step right":
        upd = True
        xr += int(-5 * math.sin(yaw))
        yr += int(5 * math.cos(yaw))
    
    #handle keyboard events
    keyb = cv2.waitKey (1)
    
    if (keyb != -1):
        upd = True
    
    if (keyb & 0xFF == ord ('q')):
        break

    elif (keyb & 0xFF == ord ('t')):
        yr -= 5
    
    elif (keyb & 0xFF == ord ('f')):
        xr -= 5
    
    elif (keyb & 0xFF == ord ('g')):
        yr += 5
    
    elif (keyb & 0xFF == ord ('h')):
        xr += 5

    elif (keyb & 0xFF == ord ('i')):
        yb -= 5
    
    elif (keyb & 0xFF == ord ('j')):
        xb -= 5
    
    elif (keyb & 0xFF == ord ('k')):
        yb += 5
    
    elif (keyb & 0xFF == ord ('l')):
        xb += 5
        
    elif (keyb & 0xFF == ord ('z')):
        yaw -= 0.1
        
    elif (keyb & 0xFF == ord ('c')):
        yaw += 0.1
        
    elif (keyb & 0xFF == ord ('w')):
        xr += int(5 * math.cos(yaw))
        yr += int(5 * math.sin(yaw))
        
    elif (keyb & 0xFF == ord ('a')):
        xr += int(5 * math.sin(yaw))
        yr += int(-5 * math.cos(yaw))
    
    elif (keyb & 0xFF == ord ('d')):
        xr += int(-5 * math.sin(yaw))
        yr += int(5 * math.cos(yaw))
    

cv2.waitKey           (1)
cv2.destroyAllWindows ()